<a href="https://colab.research.google.com/github/currencyfxjle/Monday_reports/blob/main/Monday_Reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/week_sales.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel('/content/week_sales.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Weekly sales matrices calculations
        def calculate_matrix(groupby_column):
            weekly_sales = df.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN')]).agg({'Amount': 'sum'}).reset_index()
            pivot = weekly_sales.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SUN'))

            # Insert totals at the top and label as "TOTALS"
            totals = pivot.sum().rename('TOTALS')
            pivot.loc['TOTALS', :] = totals
            ordered_index = ['TOTALS'] + [x for x in pivot.index if x != 'TOTALS']
            pivot = pivot.reindex(ordered_index)
            return pivot

        matrix_rep_df = calculate_matrix('Rep')
        matrix_item_df = calculate_matrix('Item')
        matrix_name_df = calculate_matrix('Name')

        # Save matrices for the current sheet to Excel
        matrix_rep_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Rep', index=False)
        matrix_item_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item', index=False)
        matrix_name_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Customer', index=False)

        # Autofit columns for each matrix sheet
        for suffix in ['Rep', 'Item', 'Customer']:
            worksheet = writer.sheets[f'{sheet_name} {suffix}']
            for column_cells in worksheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

invoices and credit memo

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/week_sales.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Define a function to calculate matrices for invoices and credit memos separately
    def calculate_matrix(groupby_column):
        # Read the data for the current sheet
        df = pd.read_excel('/content/week_sales.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing (same as before)
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Separate invoices and credit memos
        invoices = df[df['Type'] == 'Invoice']
        credit_memos = df[df['Type'] == 'Credit Memo']

        # Calculate matrices for invoices and credit memos
        weekly_sales_invoices = invoices.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN')]).agg({'Amount': 'sum'}).reset_index()
        weekly_sales_credit_memos = credit_memos.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN')]).agg({'Amount': 'sum'}).reset_index()

        # Pivot tables for invoices and credit memos
        pivot_invoices = weekly_sales_invoices.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SUN'))
        pivot_credit_memos = weekly_sales_credit_memos.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SUN'))

        # Insert totals and concatenate invoices and credit memos
        pivot_invoices.loc['TOTALS', :] = pivot_invoices.sum()
        pivot_credit_memos.loc['TOTALS', :] = pivot_credit_memos.sum()

        combined_matrix = pd.concat([pivot_invoices, pivot_credit_memos], axis=0, keys=['Invoices', 'Credit Memos'])

        return combined_matrix

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        matrix_combined_df = calculate_matrix('Rep')

        # Save the combined matrix for the current sheet to Excel
        matrix_combined_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Rep', index=False)

        # Autofit columns for the matrix sheet
        worksheet = writer.sheets[f'{sheet_name} Rep']
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length

        # Now, apply the same logic to 'Item' and 'Name' tabs
        matrix_combined_item_df = calculate_matrix('Item')
        matrix_combined_item_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item', index=False)

        matrix_combined_name_df = calculate_matrix('Name')
        matrix_combined_name_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Customer', index=False)

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Invoices & Credits separated by tab

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/week_sales.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel('/content/week_sales.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing (same as your existing code)
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Separate invoices and credit memos
        invoices_df = df[df['Type'] == 'Invoice']
        credit_memos_df = df[df['Type'] == 'Credit Memo']

        # Weekly sales matrices calculations for invoices and credit memos
        def calculate_matrix(groupby_column, data):
            weekly_sales = data.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN')]).agg({'Amount': 'sum'}).reset_index()
            pivot = weekly_sales.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SUN'))

            # Insert totals at the top and label as "TOTALS"
            totals = pivot.sum().rename('TOTALS')
            pivot.loc['TOTALS', :] = totals
            ordered_index = ['TOTALS'] + [x for x in pivot.index if x != 'TOTALS']
            pivot = pivot.reindex(ordered_index)
            return pivot

        matrix_rep_df_invoices = calculate_matrix('Rep', invoices_df)
        matrix_rep_df_credit_memos = calculate_matrix('Rep', credit_memos_df)
        matrix_item_df_invoices = calculate_matrix('Item', invoices_df)
        matrix_item_df_credit_memos = calculate_matrix('Item', credit_memos_df)
        matrix_name_df_invoices = calculate_matrix('Name', invoices_df)
        matrix_name_df_credit_memos = calculate_matrix('Name', credit_memos_df)

        # Create a new DataFrame combining invoices and credit memos
        combined_df = pd.concat([matrix_rep_df_invoices, matrix_rep_df_credit_memos], axis=1)

        # Save the combined DataFrame to Excel with the desired sheet name
        combined_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Combined', index=False)

        # Save matrices for items and names for invoices and credit memos
        matrix_item_df_invoices.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item Invoices', index=False)
        matrix_item_df_credit_memos.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item Credit Memos', index=False)
        matrix_name_df_invoices.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Name Invoices', index=False)
        matrix_name_df_credit_memos.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Name Credit Memos', index=False)

        # Autofit columns for all sheets
        for suffix in ['Combined', 'Item Invoices', 'Item Credit Memos', 'Name Invoices', 'Name Credit Memos']:
            worksheet = writer.sheets[f'{sheet_name} {suffix}']
            for column_cells in worksheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Test improvements invoice & credits....

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/week_sales.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel('/content/week_sales.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Weekly sales matrices calculations
        def calculate_matrix(groupby_column):
            weekly_sales = df.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN')]).agg({'Amount': 'sum'}).reset_index()
            pivot = weekly_sales.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SUN'))

            # Insert totals at the top and label as "TOTALS"
            totals = pivot.sum().rename('TOTALS')
            pivot.loc['TOTALS', :] = totals
            ordered_index = ['TOTALS'] + [x for x in pivot.index if x != 'TOTALS']
            pivot = pivot.reindex(ordered_index)
            return pivot

        matrix_rep_df = calculate_matrix('Rep')
        matrix_item_df = calculate_matrix('Item')
        matrix_name_df = calculate_matrix('Name')

        # Save matrices for the current sheet to Excel
        matrix_rep_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Rep', index=False)
        matrix_item_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item', index=False)
        matrix_name_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Customer', index=False)

        # Autofit columns for each matrix sheet
        for suffix in ['Rep', 'Item', 'Customer']:
            worksheet = writer.sheets[f'{sheet_name} {suffix}']
            for column_cells in worksheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

TOTALES de intervalos semanales

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/week_sales.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel('/content/week_sales.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Total sales calculations
        def calculate_total(groupby_column):
            return df.groupby(groupby_column).agg({'Amount': 'sum'}).reset_index()

        total_rep_df = calculate_total('Rep')
        total_item_df = calculate_total('Item')
        total_name_df = calculate_total('Name')

        # Save totals for the current sheet to Excel
        total_rep_df.to_excel(writer, sheet_name=f'{sheet_name} Rep Total', index=False)
        total_item_df.to_excel(writer, sheet_name=f'{sheet_name} Item Total', index=False)
        total_name_df.to_excel(writer, sheet_name=f'{sheet_name} Customer Total', index=False)

        # Autofit columns for each sheet
        for suffix in ['Rep Total', 'Item Total', 'Customer Total']:
            worksheet = writer.sheets[f'{sheet_name} {suffix}']
            for column_cells in worksheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

REPORTE INVENTARIO JESSICA

In [ ]:
import pandas as pd

#SEMANA DE LUNES A VIERNES
# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/Inventario_Jessica.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel('/content/Inventario_Jessica.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Weekly sales matrices calculations
        def calculate_matrix(groupby_column, values_column):
            weekly_sales = df.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN')]).agg({values_column: 'sum'}).reset_index()
            pivot = weekly_sales.pivot_table(values=values_column, index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SUN'))

            # Insert totals at the top and label as "TOTALS"
            totals = pivot.sum().rename('TOTALS')
            pivot.loc['TOTALS', :] = totals
            ordered_index = ['TOTALS'] + [x for x in pivot.index if x != 'TOTALS']
            pivot = pivot.reindex(ordered_index)
            return pivot

        matrix_rep_df = calculate_matrix('Rep', 'Amount')
        matrix_item_amount_df = calculate_matrix('Item', 'Amount')
        matrix_item_qty_df = calculate_matrix('Item', 'Qty')
        matrix_name_df = calculate_matrix('Name', 'Amount')

        # Save matrices for the current sheet to Excel
        matrix_rep_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Rep', index=False)
        matrix_item_amount_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item Amount', index=False)
        matrix_item_qty_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Item Qty', index=False)
        matrix_name_df.reset_index().to_excel(writer, sheet_name=f'{sheet_name} Customer', index=False)

        # Autofit columns for each matrix sheet
        for suffix in ['Rep', 'Item Amount', 'Item Qty', 'Customer']:
            worksheet = writer.sheets[f'{sheet_name} {suffix}']
            for column_cells in worksheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

QTY & AMOUNT SEPARATED

In [ ]:
import pandas as pd
# SEMANA DOMINGO A SABADO
# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/Jessica_inventory_code.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel('/content/Jessica_inventory_code.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Weekly sales matrices calculations
        def calculate_matrix(groupby_column):
            weekly_sales = df.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SAT')]).agg({'Amount': 'sum', 'Qty': 'sum'}).reset_index()
            pivot_amount = weekly_sales.pivot_table(values='Amount', index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SAT'))
            pivot_qty = weekly_sales.pivot_table(values='Qty', index=groupby_column, columns=pd.Grouper(key='Date', freq='W-SAT'))

            # Insert totals at the top and label as "TOTALS"
            for pivot in [pivot_amount, pivot_qty]:
                totals = pivot.sum().rename('TOTALS')
                pivot.loc['TOTALS', :] = totals
                ordered_index = ['TOTALS'] + [x for x in pivot.index if x != 'TOTALS']
                pivot = pivot.reindex(ordered_index)

            # Concatenate Amount and Qty side by side
            result = pd.concat([pivot_amount, pivot_qty], axis=1, keys=['Amount', 'Qty'])

            return result

        matrix_rep_df = calculate_matrix('Rep')
        matrix_item_df = calculate_matrix('Item')
        matrix_name_df = calculate_matrix('Name')

        # Save matrices for the current sheet to Excel
        matrix_rep_df.to_excel(writer, sheet_name=f'{sheet_name} Rep', index=True)
        matrix_item_df.to_excel(writer, sheet_name=f'{sheet_name} Item', index=True)
        matrix_name_df.to_excel(writer, sheet_name=f'{sheet_name} Customer', index=True)

# Autofit columns for each matrix sheet
for suffix in ['Rep', 'Item', 'Customer']:
    worksheet = writer.sheets[f'{sheet_name} {suffix}']
    for column_cells in worksheet.columns:
        try:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length
        except AttributeError:
            continue


# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

QTY & AMOUNT BY WEEK ALONG SIDE

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/CA_items.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel('/content/CA_items.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Weekly sales matrices calculations
        def calculate_matrix(groupby_column):
            weekly_sales = df.groupby([groupby_column, pd.Grouper(key='Date', freq='W-SUN')]).agg({'Amount': 'sum', 'Qty': 'sum'}).reset_index()

            # Create multi-level columns
            multi_indexed_df = weekly_sales.set_index([groupby_column, 'Date']).stack().unstack(level=-1).unstack(level=-1)
            multi_indexed_df = multi_indexed_df.swaplevel(i=0, j=1, axis=1).sort_index(axis=1)

            # Insert totals
            totals = multi_indexed_df.sum().rename('TOTALS')
            multi_indexed_df.loc['TOTALS', :] = totals
            ordered_index = ['TOTALS'] + [x for x in multi_indexed_df.index if x != 'TOTALS']
            multi_indexed_df = multi_indexed_df.reindex(ordered_index)

            return multi_indexed_df

        matrix_rep_df = calculate_matrix('Rep')
        matrix_item_df = calculate_matrix('Item')
        matrix_name_df = calculate_matrix('Name')

        # Save matrices for the current sheet to Excel
        matrix_rep_df.to_excel(writer, sheet_name=f'{sheet_name} Rep', index=True)
        matrix_item_df.to_excel(writer, sheet_name=f'{sheet_name} Item', index=True)
        matrix_name_df.to_excel(writer, sheet_name=f'{sheet_name} Customer', index=True)

    # Autofit columns for each matrix sheet
    for suffix in ['Rep', 'Item', 'Customer']:
        worksheet = writer.sheets[f'{sheet_name} {suffix}']
        for column_cells in worksheet.columns:
            try:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length
            except AttributeError:
                continue

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/CA_items.xlsx')

# Create an ExcelWriter to save the results
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:

    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel('/content/CA_items.xlsx', sheet_name=sheet_name, header=0)

        # Preprocessing
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Item'] = df['Item'].astype(str)
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)
        df['Qty'] = df['Qty'].replace(',', '', regex=True).astype(float)

        # Total sales calculations
        def calculate_totals(groupby_column):
            totals_df = df.groupby(groupby_column).agg({'Amount': 'sum', 'Qty': 'sum'}).reset_index()

            # Insert overall totals row
            overall_totals = totals_df[['Amount', 'Qty']].sum()
            overall_totals[groupby_column] = 'TOTALS'
            totals_df = pd.concat([totals_df, pd.DataFrame([overall_totals])], ignore_index=True)
            ordered_index = ['TOTALS'] + [x for x in totals_df[groupby_column] if x != 'TOTALS']
            totals_df = totals_df.set_index(groupby_column).loc[ordered_index].reset_index()

            return totals_df

        matrix_rep_df = calculate_totals('Rep')
        matrix_item_df = calculate_totals('Item')
        matrix_name_df = calculate_totals('Name')

        # Save matrices for the current sheet to Excel
        matrix_rep_df.to_excel(writer, sheet_name=f'{sheet_name} Rep', index=False)
        matrix_item_df.to_excel(writer, sheet_name=f'{sheet_name} Item', index=False)
        matrix_name_df.to_excel(writer, sheet_name=f'{sheet_name} Customer', index=False)

    # Autofit columns for each matrix sheet
    for suffix in ['Rep', 'Item', 'Customer']:
        worksheet = writer.sheets[f'{sheet_name} {suffix}']
        for column_cells in worksheet.columns:
            try:
                length = max(len(str(cell.value)) for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = length
            except AttributeError:
                continue

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Migue brand

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/Fresno_report.xlsx')

# Create a new Excel writer object
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:
    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        # Read the current tab as a DataFrame
        df = pd.read_excel('/content/Fresno_report.xlsx', sheet_name=sheet_name, header=0)

        # Convert 'Type', 'Name', and 'Date' columns to appropriate datatypes
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])

        # Clean 'Amount' column by removing commas and converting to float
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)

        # Function to create pivot data
        def create_pivot_data(dataframe):
            pivot_data = dataframe.pivot_table(values='Amount', index='Name', columns=pd.Grouper(key='Date', freq='W-SUN'), aggfunc='sum').reset_index()

            # Calculate totals and append to the top
            data_totals = pivot_data.sum(numeric_only=True)
            data_totals['Name'] = 'TOTAL'
            pivot_with_totals = pd.concat([pd.DataFrame(data_totals).T, pivot_data], ignore_index=True)

            # Ensure 'Name' column is the first column
            ordered_cols = ['Name'] + [col for col in pivot_with_totals if col != 'Name']
            pivot_with_totals = pivot_with_totals[ordered_cols]

            return pivot_with_totals

        # Separate data by Type and generate pivot tables
        invoices_data = create_pivot_data(df[df['Type'] == 'Invoice'])
        credit_memos_data = create_pivot_data(df[df['Type'] == 'Credit Memo'])

        # Write the separated data to Excel
        invoices_data.to_excel(writer, sheet_name=sheet_name + ' - Invoices', index=False)
        credit_memos_data.to_excel(writer, sheet_name=sheet_name + ' - Credit Memos', index=False)

    # Autofit columns for each sheet
    workbook = writer.book
    for sheet in workbook.sheetnames:
        worksheet = workbook[sheet]
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/Fresno_report.xlsx')

# Create a new Excel writer object
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:
    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        # Read the current tab as a DataFrame
        df = pd.read_excel('/content/Fresno_report.xlsx', sheet_name=sheet_name, header=0)

        # Convert 'Type', 'Name', and 'Date' columns to appropriate datatypes
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])

        # Clean 'Amount' column by removing commas and converting to float
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)

        # Function to create pivot data
        def create_pivot_data(dataframe):
            pivot_data = dataframe.pivot_table(values='Amount', index='Name', columns=pd.Grouper(key='Date', freq='W-SUN'), aggfunc='sum').reset_index()

            # Calculate totals and append to the top
            data_totals = pivot_data.sum(numeric_only=True)
            data_totals['Name'] = 'TOTAL'
            pivot_with_totals = pd.concat([pd.DataFrame(data_totals).T, pivot_data], ignore_index=True)

            # Ensure 'Name' column is the first column
            ordered_cols = ['Name'] + [col for col in pivot_with_totals if col != 'Name']
            pivot_with_totals = pivot_with_totals[ordered_cols]

            return pivot_with_totals

        # Separate data by Type and generate pivot tables
        invoices_data = create_pivot_data(df[df['Type'] == 'Invoice'])
        credit_memos_data = create_pivot_data(df[df['Type'] == 'Credit Memo'])

        # Write the separated data to Excel
        invoices_data.to_excel(writer, sheet_name=sheet_name + ' - Invoices', index=False)
        credit_memos_data.to_excel(writer, sheet_name=sheet_name + ' - Credit Memos', index=False)

    # Autofit columns for each sheet
    workbook = writer.book
    for sheet in workbook.sheetnames:
        worksheet = workbook[sheet]
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')

Ventas frecuencia mensual

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/Cardenas_2023.xlsx')

# Create a new Excel writer object
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:
    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        # Read the current tab as a DataFrame
        df = pd.read_excel('/content/Cardenas_2023.xlsx', sheet_name=sheet_name, header=0)

        # Convert 'Type', 'Name', and 'Date' columns to appropriate datatypes
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])

        # Clean 'Amount' column by removing commas and converting to float
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)

        # Function to create pivot data
        def create_pivot_data(dataframe):
            pivot_data = dataframe.pivot_table(values='Amount', index='Name', columns=pd.Grouper(key='Date', freq='M'), aggfunc='sum').reset_index()

            # Calculate totals and append to the top
            data_totals = pivot_data.sum(numeric_only=True)
            data_totals['Name'] = 'TOTAL'
            pivot_with_totals = pd.concat([pd.DataFrame(data_totals).T, pivot_data], ignore_index=True)

            # Ensure 'Name' column is the first column
            ordered_cols = ['Name'] + [col for col in pivot_with_totals if col != 'Name']
            pivot_with_totals = pivot_with_totals[ordered_cols]

            return pivot_with_totals

        # Separate data by Type and generate pivot tables
        invoices_data = create_pivot_data(df[df['Type'] == 'Invoice'])
        credit_memos_data = create_pivot_data(df[df['Type'] == 'Credit Memo'])

        # Write the separated data to Excel
        invoices_data.to_excel(writer, sheet_name=sheet_name + ' - Invoices', index=False)
        credit_memos_data.to_excel(writer, sheet_name=sheet_name + ' - Credit Memos', index=False)

    # Autofit columns for each sheet
    workbook = writer.book
    for sheet in workbook.sheetnames:
        worksheet = workbook[sheet]
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Suma de invoice + credit

In [ ]:
import pandas as pd

# Read the Excel file with multiple tabs
excel_file = pd.ExcelFile('/content/Fresno_report.xlsx')

# Create a new Excel writer object
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:
    # Iterate over each tab in the Excel file
    for sheet_name in excel_file.sheet_names:
        # Read the current tab as a DataFrame
        df = pd.read_excel('/content/Fresno_report.xlsx', sheet_name=sheet_name, header=0)

        # Convert 'Type', 'Name', and 'Date' columns to appropriate datatypes
        df['Type'] = df['Type'].astype(str)
        df['Name'] = df['Name'].astype(str)
        df['Date'] = pd.to_datetime(df['Date'])

        # Clean 'Amount' column by removing commas and converting to float
        df['Amount'] = df['Amount'].replace(',', '', regex=True).astype(float)

        # Function to create pivot data
        def create_pivot_data(dataframe):
            pivot_data = dataframe.pivot_table(values='Amount', index='Name', columns=pd.Grouper(key='Date', freq='M'), aggfunc='sum').reset_index()

            # Calculate totals and append to the top
            data_totals = pivot_data.sum(numeric_only=True)
            data_totals['Name'] = 'TOTAL'
            pivot_with_totals = pd.concat([pd.DataFrame(data_totals).T, pivot_data], ignore_index=True)

            # Ensure 'Name' column is the first column
            ordered_cols = ['Name'] + [col for col in pivot_with_totals if col != 'Name']
            pivot_with_totals = pivot_with_totals[ordered_cols]

            return pivot_with_totals

        # Generate a pivot table for all data
        all_data = create_pivot_data(df)

        # Write the combined data to Excel
        all_data.to_excel(writer, sheet_name=sheet_name + ' - Monthly Sales', index=False)

    # Autofit columns for each sheet
    workbook = writer.book
    for sheet in workbook.sheetnames:
        worksheet = workbook[sheet]
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length

# Download the Excel file
from google.colab import files
files.download('output_file.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>